<a href="https://colab.research.google.com/github/mherskovitz/FRED/blob/main/Fred_Corp_OAS_Yld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Script to Retrieve, Plot and Download Corporate Yield and OAS from FRED

Downloads will be both PNG files and html files to load in the browser

Requires that you have a FRED API

1. Install necessary libraries

In [1]:
!pip install fredapi
!pip install plotly
!pip install kaleido
!pip install pandas_datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.0 MB/s eta 0:00:00


Load Packages

In [2]:
import pandas as pd
import pandas_datareader.data as web
import plotly.express as px
import plotly.graph_objects as go
import datetime
from fredapi import Fred
from google.colab import userdata  # Import userdata to access Google Secrets
from google.colab import files
import io
import os

Set up FRED API Key

In [4]:
# Access the FRED API key from Google Secrets
try:
    FRED_API_KEY = userdata.get('FRED_API')  # Replace 'FRED_API_KEY' with your secret name
except Exception as e:
    raise Exception("Failed to retrieve FRED API key from Google Secrets. Ensure the secret is set up correctly.") from e

# Initialize FRED API client
fred = Fred(api_key=FRED_API_KEY)

Functions to Retrieve and Plot Data

In [5]:
# Initialize FRED API client


# Function to fetch data and metadata from FRED
def fetch_fred_data(series_id, start_date, end_date):
    """
    Fetch data and metadata from FRED using the series ID.

    Parameters:
    - series_id: str, the FRED series ID (e.g., 'BAMLH0A0HYM2')
    - start_date: str, the start date in 'YYYY-MM-DD' format
    - end_date: str, the end date in 'YYYY-MM-DD' format

    Returns:
    - pd.DataFrame, the fetched data
    - str, the series name (title)
    """
    # Fetch data using pandas_datareader
    data = web.DataReader(series_id, 'fred', start_date, end_date)

    # Fetch metadata using fredapi
    series_info = fred.get_series_info(series_id)
    series_name = series_info.get('title', series_id)  # Use series ID as fallback if title is missing

    return data, series_name

In [6]:
# Function to plot the data using Plotly

def plot_fred_data(series_ids, output_dir='plots'):
    """
    Plot the FRED data using Plotly for multiple series.

    Parameters:
    - series_ids: list, list of FRED series IDs to plot
    - output_dir: str, directory to save plot files (default: 'plots')
    """
    # Get the current date and calculate the start date (10 years ago)
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=10 * 365)

    # Create output directory in Colab
    os.makedirs(output_dir, exist_ok=True)

    # Create a figure for each series
    for series_id in series_ids:
        # Fetch the data and series name
        data, series_name = fetch_fred_data(
            series_id, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')
        )

        # Drop missing values
        data = data.dropna()

        # Calculate the average and standard deviation
        avg_rate = data[series_id].mean()
        std_dev = data[series_id].std()

        # Create the plot
        fig = px.line(data, x=data.index, y=series_id, title=series_name)

        # Add horizontal lines for average and standard deviation
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=[avg_rate] * len(data),
                mode='lines',
                name='Average Rate',
                line=dict(color='red', dash='dash'),
            )
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=[avg_rate + std_dev] * len(data),
                mode='lines',
                name='+1 Std Dev',
                line=dict(color='green', dash='dash'),
            )
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=[avg_rate - std_dev] * len(data),
                mode='lines',
                name='-1 Std Dev',
                line=dict(color='green', dash='dash'),
            )
        )

        # Update layout
        fig.update_layout(
            xaxis_title='Date', yaxis_title='Spread (%)', showlegend=True
        )

        # Generate filenames
        html_filename = f"{series_id}.html"
        png_filename = f"{series_id}.png"

        # Create full paths
        html_filepath = os.path.join(output_dir, html_filename)
        png_filepath = os.path.join(output_dir, png_filename)

        # Save the plots
        fig.write_html(html_filepath)
        fig.write_image(png_filepath)

        # Show the plot in notebook
        fig.show()

        # Download files automatically
        files.download(html_filepath)
        files.download(png_filepath)

def plot_dual_axis_fred_data(yield_series_id, spread_series_id, output_dir='plots'):
    """
    Plot two FRED series on the same graph with dual y-axes and save to file in Colab.

    Parameters:
    - yield_series_id: str, FRED series ID for yield data (left axis)
    - spread_series_id: str, FRED series ID for spread data (right axis)
    - output_dir: str, directory to save plot files (default: 'plots')
    """
    # Create output directory in Colab
    os.makedirs(output_dir, exist_ok=True)

    # Get the current date and calculate the start date (10 years ago)
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=10*365)

    # Fetch both datasets
    yield_data, yield_name = fetch_fred_data(yield_series_id, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    spread_data, spread_name = fetch_fred_data(spread_series_id, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

    # Combine the datasets and align dates
    combined_data = pd.concat([
        yield_data[yield_series_id],
        spread_data[spread_series_id]
    ], axis=1).dropna()

    # Create figure with secondary y-axis
    fig = go.Figure()

    # Add yield trace on left y-axis
    fig.add_trace(
        go.Scatter(
            x=combined_data.index,
            y=combined_data[yield_series_id],
            name=yield_name,
            line=dict(color='blue')
        )
    )

    # Add spread trace on right y-axis
    fig.add_trace(
        go.Scatter(
            x=combined_data.index,
            y=combined_data[spread_series_id],
            name=spread_name,
            line=dict(color='red'),
            yaxis='y2'
        )
    )

    # Calculate statistics for both series
    yield_avg = combined_data[yield_series_id].mean()
    yield_std = combined_data[yield_series_id].std()
    spread_avg = combined_data[spread_series_id].mean()
    spread_std = combined_data[spread_series_id].std()

    # Add statistics lines for yield (left axis)
    fig.add_trace(
        go.Scatter(
            x=combined_data.index,
            y=[yield_avg] * len(combined_data),
            name='Yield Average',
            line=dict(color='blue', dash='dash'),
            opacity=0.5
        )
    )

    # Add statistics lines for spread (right axis)
    fig.add_trace(
        go.Scatter(
            x=combined_data.index,
            y=[spread_avg] * len(combined_data),
            name='Spread Average',
            line=dict(color='red', dash='dash'),
            opacity=0.5,
            yaxis='y2'
        )
    )

    # Update layout with dual axes
    fig.update_layout(
        title=f'Comparison of {yield_name} vs {spread_name}',
        xaxis_title='Date',
        yaxis=dict(
            title='Yield (%)',
            titlefont=dict(color='blue'),
            tickfont=dict(color='blue')
        ),
        yaxis2=dict(
            title='Spread (bps)',
            titlefont=dict(color='red'),
            tickfont=dict(color='red'),
            overlaying='y',
            side='right'
        ),
        showlegend=True,
        legend=dict(
            yanchor='top',
            y=0.99,
            xanchor='left',
            x=0.01
        )
    )

    # Generate filenames
    html_filename = f"comparison_{yield_series_id}_vs_{spread_series_id}.html"
    png_filename = f"comparison_{yield_series_id}_vs_{spread_series_id}.png"

    # Create full paths
    html_filepath = os.path.join(output_dir, html_filename)
    png_filepath = os.path.join(output_dir, png_filename)

    # Save the plots
    fig.write_html(html_filepath)
    fig.write_image(png_filepath)

    # Show the plot in notebook
    fig.show()

    # Download files automatically
    files.download(html_filepath)
    files.download(png_filepath)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Main Script

In [7]:
# Main script
if __name__ == "__main__":
    # Define output directory
    output_dir = "fred_plots"

    # Define the FRED series IDs for individual plots
    series_ids = [
        'BAMLH0A0HYM2',  # ICE BofA US High Yield Index Option-Adjusted Spread
        'BAMLC0A0CMEY',  # ICE BofA US Corporate Master Effective Yield
        'BAMLC0A3CAEY'   # ICE BofA US Corporate 3-5 Year Effective Yield
    ]

    # Plot individual series and save to files
    plot_fred_data(series_ids, output_dir=output_dir)

    # Plot dual-axis comparison
    plot_dual_axis_fred_data(
        yield_series_id='BAMLC0A0CMEY',  # Corporate Master Effective Yield
        spread_series_id='BAMLH0A0HYM2',  # High Yield OAS
        output_dir=output_dir
    )
    plot_dual_axis_fred_data(
        yield_series_id='BAMLC0A0CMEY',  # Investment grade Effective Yield
        spread_series_id='BAMLC0A0CM',  # Investment grade OAS
        output_dir=output_dir
    )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>